In [ ]:
import tensorflow as tf
import argparse
import os.path
from cifar10_models import *
from cifar10_input import *
from utils import *
import itertools
import logging
import pickle

tf.logging.set_verbosity(tf.logging.INFO)

# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler('tensorflow.log')
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
log.addHandler(fh)

#### SET THIS BEFORE RUNNING
DIR = "cifar10"
NAME = "cifar10_scm_{}_{}_{}_{}"
PATH = os.path.join(DIR,NAME)

def make_dirs(name): 
    try:
        os.makedirs(name)
    except FileExistsError:
        pass

def load_checkpoints(name):
    try:
        checkpoints = sorted(["{}/{}".format(ROOT.format(name),x[:-5]) for x in os.listdir(ROOT.format(name)) if x.endswith(".meta")], 
                     key=lambda x: int(x.split("-")[-1]))
    except FileNotFoundError:
        checkpoints = []
    return checkpoints

def train(model, name, restore_point = None, wd1=0., wd2=0., wd3=0., drop1=1., save=True):
    with tf.Session() as sess:
        model._restore_or_init(sess, restore_point)
        model.weight_decay_layer_1.assign(wd1)
        model.weight_decay_layer_2.assign(wd2)
        model.weight_decay_layer_3.assign(wd3)
        model.drop_fc1.assign(drop1)
        
        try:
            while True:
                sess.run(model.train_step)
        except tf.errors.OutOfRangeError as e:
            pass
        
        global_step = model.global_step.eval()
        if save:
            checkpoint_name = CHECKPOINT.format(name) + "_" + str(global_step).rjust(10,"0")
            tf.logging.info("Saving to: " + checkpoint_name)
            model.saver.save(sess, checkpoint_name)
    return checkpoint_name
        
def evaluate(model, restore_point = None):
    acc = []
    with tf.Session() as sess:
        model._restore_or_init(sess, restore_point)
        try:
            while True:
                acc.append(model.accuracy.eval())
        except tf.errors.OutOfRangeError as e:
            pass
    return np.mean(acc)

@np.vectorize
def nplte(x,y):
    return x<=y

def train_to_convergence(train_files, wd1=0., wd2=0., wd3=0., drop1=1., save_rate=10):
    acc = []
    checkpoint = None
    name = PATH.format(wd1,wd2,wd3,drop1)
    make_dirs(CHECKPOINT.format(name))
    i = 0
    while len(acc)<30 or nplte(np.array(acc[-20:]) - np.array(acc[-21:-1]),0).sum() < 13: # until the accuracy is just as likely to go down as up
        tf.reset_default_graph()
        train_data = Cifar10Record(train_files, epochs=1)
        model = Cifar10ShallowConvolutionalModel(train_data.image, train_data.label)
        save = not i%save_rate
        checkpoint = train(model, name, restore_point = checkpoint, wd1=wd1, wd2=wd2, wd3=wd3, drop1=drop1, save=save)
        tf.reset_default_graph()
        train_data = Cifar10Record(train_files, epochs=1)
        model = Cifar10ShallowConvolutionalModel(train_data.image, train_data.label)
        acc.append(evaluate(model, restore_point = checkpoint))
        tf.logging.info(acc[-21:])
    return acc
        
def train_a_bunch_of_cifar10s():
    wd1_space = np.linspace(0.,0.012,3) # they use 0.004 in the tensorflow example
    wd2_space = np.linspace(0.,0.012,3)
    wd3_space = np.linspace(0.,0.012,3)
    drop1_space = np.linspace(0.2,0.8,3)
    wd_drop_grid = itertools.product(wd1_space, wd2_space, wd3_space, drop1_space)
    train_files, test_files, validation_files = maybe_download_and_extract()
    try:
        with open("completed_cifar10.p",'rb') as f:
            completed = pickle.load(f)
    except:
        completed = []
    i = 0
    for wd1,wd2,wd3,drop1 in wd_drop_grid:
        if i < len(completed): #skip values until we get to a new one
            i+=1
            continue
        acc = train_to_convergence(train_files,wd1,wd2,wd3,drop1)
        tf.logging.info(acc[-10:])
        completed.append((wd1,wd2,wd3,drop1,acc))
        i+=1
        with open("completed_cifar10.p","wb") as f:
            pickle.dump(completed,f)

In [ ]:
make_dirs(ROOT.format(DIR))
train_a_bunch_of_cifar10s()

INFO:tensorflow:Saving to: ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000000400
INFO:tensorflow:../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000000400 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000000400
INFO:tensorflow:[0.35025]
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000000400
INFO:tensorflow:Saving to: ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000000800
INFO:tensorflow:../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000000800 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000000800
INFO:tensorflow:[0.35025, 0.403]
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000000800
INFO:tensorflow:Saving to: 

INFO:tensorflow:Saving to: ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000006400
INFO:tensorflow:../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000006400 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000006400
INFO:tensorflow:[0.35025, 0.403, 0.428, 0.45529997, 0.49089995, 0.506625, 0.52, 0.5392, 0.543825, 0.556875, 0.56014997, 0.570875, 0.581275, 0.587175, 0.59135, 0.5996]
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000006400
INFO:tensorflow:Saving to: ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000006800
INFO:tensorflow:../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000006800 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000006800
INFO:tensorflow:[0.35025, 0.4

INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000011200
INFO:tensorflow:[0.5392, 0.543825, 0.556875, 0.56014997, 0.570875, 0.581275, 0.587175, 0.59135, 0.5996, 0.60245, 0.60784996, 0.613825, 0.62262505, 0.627475, 0.6317, 0.63222504, 0.64185, 0.631575, 0.646175, 0.64605, 0.65065]
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000011200
INFO:tensorflow:Saving to: ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000011600
INFO:tensorflow:../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000011600 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000011600
INFO:tensorflow:[0.543825, 0.556875, 0.56014997, 0.570875, 0.581275, 0.587175, 0.59135, 0.5996, 0.60245, 0.60784996, 0.613825, 0.62262505, 0.627475, 0.6317, 0.63222504, 0.64185, 0.631575, 0.646175, 0.64605, 0.

INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000016000
INFO:tensorflow:[0.62262505, 0.627475, 0.6317, 0.63222504, 0.64185, 0.631575, 0.646175, 0.64605, 0.65065, 0.65825, 0.65810007, 0.65322495, 0.664525, 0.67095, 0.669975, 0.67012495, 0.67002505, 0.67947495, 0.682475, 0.68425006, 0.692625]
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000016000
INFO:tensorflow:Saving to: ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000016400
INFO:tensorflow:../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000016400 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000016400
INFO:tensorflow:[0.627475, 0.6317, 0.63222504, 0.64185, 0.631575, 0.646175, 0.64605, 0.65065, 0.65825, 0.65810007, 0.65322495, 0.664525, 0.67095, 0.669975, 0.67012495, 0.67002505, 0.67947495, 0.682

In [4]:
completed=[]
np.save("blah.npy", completed)